<a href="https://colab.research.google.com/github/ANUPRIYAR/HackerEarth_submissions/blob/main/ExhibitArt_Shipping_Costipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
# !unzip /content/drive/MyDrive/Deep_learning/Datasets/7342422066d511eb.zip

In [161]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn import metrics
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [85]:
train_df = pd.read_csv('/content/dataset/train.csv')
test_df = pd.read_csv('/content/dataset/test.csv')
train_df.head()

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location,Cost
0,fffe3900350033003300,Billy Jenkins,0.26,17.0,6.0,4128.0,Brass,13.91,16.27,Yes,Yes,No,Airways,No,Working Class,No,06/07/15,06/03/15,"New Michelle, OH 50777",-283.29
1,fffe3800330031003900,Jean Bryant,0.28,3.0,3.0,61.0,Brass,6.83,15.00,No,No,No,Roadways,No,Working Class,No,03/06/17,03/05/17,"New Michaelport, WY 12072",-159.96
2,fffe3600370035003100,Laura Miller,0.07,8.0,5.0,237.0,Clay,4.96,21.18,No,No,No,Roadways,Yes,Working Class,Yes,03/09/15,03/08/15,"Bowmanshire, WA 19241",-154.29
3,fffe350031003300,Robert Chaires,0.12,9.0,NaN,NaN,Aluminium,5.81,16.31,No,No,No,NaN,No,Wealthy,Yes,05/24/15,05/20/15,"East Robyn, KY 86375",-161.16
4,fffe3900320038003400,Rosalyn Krol,0.15,17.0,6.0,324.0,Aluminium,3.18,11.94,Yes,Yes,Yes,Airways,No,Working Class,No,12/18/16,12/14/16,"Aprilside, PA 52793",-159.23


In [86]:
test_df.head()

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location
0,fffe3400310033003300,James Miller,0.35,53.0,18.0,871.0,Wood,5.98,19.11,Yes,Yes,No,Airways,No,Working Class,No,07/03/17,07/06/17,"Santoshaven, IA 63481"
1,fffe3600350035003400,Karen Vetrano,0.67,7.0,4.0,108.0,Clay,6.92,13.96,No,No,No,Roadways,Yes,Working Class,No,05/02/16,05/02/16,"Ericksonton, OH 98253"
2,fffe3700360030003500,Roseanne Gaona,0.61,6.0,5.0,97.0,Aluminium,4.23,13.62,Yes,No,No,Airways,No,Working Class,No,01/04/18,01/06/18,APO AP 83453
3,fffe350038003600,Todd Almanza,0.14,15.0,8.0,757.0,Clay,6.28,23.79,No,Yes,No,Roadways,Yes,Wealthy,No,09/14/17,09/17/17,"Antonioborough, AL 54778"
4,fffe3500390032003500,Francis Rivero,0.63,10.0,4.0,1673.0,Marble,4.39,17.83,No,Yes,Yes,Roadways,No,Working Class,Yes,12/03/17,12/02/17,"Lake Frances, LA 03040"


In [87]:
train_original = train_df.copy()
test_original = test_df.copy()

In [88]:
train_df.drop(['Customer Id','Artist Name'], axis=1, inplace=True)
test_df.drop(['Customer Id','Artist Name'], axis=1, inplace=True)

In [89]:
def divide_variables(df):
  cols = df.columns
  categorical_cols = []
  numerical_cols = []
  for col in cols:
    if df[col].dtypes =='float64':
      numerical_cols.append(col)
    else: 
      categorical_cols.append(col)
  return categorical_cols, numerical_cols

In [90]:
cat_cols, numeric_cols = divide_variables(train_df)
print("Categorical Columns:", cat_cols)
print("Numerical Columns:" , numeric_cols)

Categorical Columns: ['Material', 'International', 'Express Shipment', 'Installation Included', 'Transport', 'Fragile', 'Customer Information', 'Remote Location', 'Scheduled Date', 'Delivery Date', 'Customer Location']
Numerical Columns: ['Artist Reputation', 'Height', 'Width', 'Weight', 'Price Of Sculpture', 'Base Shipping Price', 'Cost']


# **Handling Missing variables**

In [91]:
train_df.isnull().sum()

Artist Reputation         750
Height                    375
Width                     584
Weight                    587
Material                  764
Price Of Sculpture          0
Base Shipping Price         0
International               0
Express Shipment            0
Installation Included       0
Transport                1392
Fragile                     0
Customer Information        0
Remote Location           771
Scheduled Date              0
Delivery Date               0
Customer Location           0
Cost                        0
dtype: int64

In [92]:
test_df.isnull().sum()

Artist Reputation        222
Height                   119
Width                    141
Weight                   149
Material                   0
Price Of Sculpture         0
Base Shipping Price        0
International              0
Express Shipment           0
Installation Included      0
Transport                232
Fragile                    0
Customer Information       0
Remote Location            0
Scheduled Date             0
Delivery Date              0
Customer Location          0
dtype: int64

In [93]:
# Filling na values for categorical variables
for col in cat_cols:
  train_df[col].fillna(train_df[col].mode()[0], inplace=True)

for col in cat_cols:
  test_df[col].fillna(test_df[col].mode()[0], inplace=True)

In [94]:
# Removing cost from numerical Variables
numeric_cols.remove('Cost')

In [95]:
# Filling na values for Numeric variables
for col in numeric_cols:
  train_df[col].fillna(train_df[col].median(), inplace=True)

for col in numeric_cols:
  test_df[col].fillna(test_df[col].median(), inplace=True)

In [96]:
# Sanity check for Null Values
train_df.isnull().sum().any() & test_df.isnull().sum().any()

False

# **Data Preprocessing**

# Preprocessing Categorical Variables



In [97]:
def preprocess_category(df, categorical_fields):
  encoding = LabelEncoder()
  for col in categorical_fields:
    df[col] = encoding.fit_transform(df[col])
  return df


In [98]:
train_df_processed = preprocess_category(train_df,cat_cols[:-3])
test_df_processed = preprocess_category(test_df,cat_cols[:-3])

In [99]:
Material = pd.get_dummies(train_df_processed['Material'], drop_first=True, prefix='Material')

In [100]:
Material_test = pd.get_dummies(test_df_processed['Material'], drop_first=True, prefix='Material')

In [102]:
Transport = pd.get_dummies(train_df_processed['Transport'], drop_first=True, prefix='Transport')
Transport_test = pd.get_dummies(test_df_processed['Transport'], drop_first=True, prefix='Transport')

In [103]:
# Joining all together
train_df_processed = pd.concat([train_df_processed.drop(['Material', 'Transport'], axis=1), Material, Transport], axis=1)
test_df_processed = pd.concat([test_df_processed.drop(['Material', 'Transport'], axis=1) ,Material_test,  Transport_test], axis=1)

# **Extracting state field**

In [106]:
# Extracting states from Customer Location
train_df_processed['state'] = train_df_processed['Customer Location'].str.split(',').str[1].str.split(' ').str[1]
train_df_processed.drop('Customer Location', axis=1, inplace=True)

In [108]:
test_df_processed['state'] = test_df_processed['Customer Location'].str.split(',').str[1].str.split(' ').str[1]
test_df_processed.drop('Customer Location', axis=1, inplace=True)

In [111]:
train_df_processed.head(2)

,Artist Reputation,Height,Width,Weight,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Cost,Material_1,Material_2,Material_3,Material_4,Material_5,Material_6,Transport_1,Transport_2,state
0,0.26,17.0,6.0,4128.0,13.91,16.27,1,1,0,0,1,0,06/07/15,06/03/15,-283.29,1,0,0,0,0,0,0,0,OH
1,0.28,3.0,3.0,61.0,6.83,15.00,0,0,0,0,1,0,03/06/17,03/05/17,-159.96,1,0,0,0,0,0,1,0,WY


In [115]:
encoding = LabelEncoder()
train_df_processed['state'] = encoding.fit_transform(train_df_processed['state'].astype('str'))
test_df_processed['state'] = encoding.fit_transform(test_df_processed['state'].astype('str'))

# ***Preprocessing Date fields***

In [118]:
# Transforming Date fields 
def extracting_datediff(df):
  df['Scheduled Date'] = pd.to_datetime(df['Scheduled Date'])
  df['Delivery Date'] = pd.to_datetime(df['Delivery Date'])
  df['date_diff'] = (df['Scheduled Date'] - df['Delivery Date'])
  df['date_diff'] = df['date_diff'].astype(str).str.split().str[0]
  df['date_diff'] = pd.to_numeric(df['date_diff'])
  df.drop('Scheduled Date', axis=1, inplace=True)
  return df


In [119]:
train_df_processed = extracting_datediff(train_df_processed)
train_df_processed.head(2)

,Artist Reputation,Height,Width,Weight,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Fragile,Customer Information,Remote Location,Delivery Date,Cost,Material_1,Material_2,Material_3,Material_4,Material_5,Material_6,Transport_1,Transport_2,state,date_diff
0,0.26,17.0,6.0,4128.0,13.91,16.27,1,1,0,0,1,0,2015-06-03,-283.29,1,0,0,0,0,0,0,0,35,4
1,0.28,3.0,3.0,61.0,6.83,15.00,0,0,0,0,1,0,2017-03-05,-159.96,1,0,0,0,0,0,1,0,50,1


In [120]:
test_df_processed = extracting_datediff(test_df_processed)
test_df_processed.head(2)

,Artist Reputation,Height,Width,Weight,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Fragile,Customer Information,Remote Location,Delivery Date,Material_1,Material_2,Material_3,Material_4,Material_5,Material_6,Transport_1,Transport_2,state,date_diff
0,0.35,53.0,18.0,871.0,5.98,19.11,1,1,0,0,1,0,2017-07-06,0,0,0,0,0,1,0,0,12,-3
1,0.67,7.0,4.0,108.0,6.92,13.96,0,0,0,1,1,0,2016-05-02,0,0,1,0,0,0,1,0,35,0


In [124]:
# Extracting values from date fields 
def extract_date(df):
  df['dt_day'] = df['Delivery Date'].dt.day
  df['dt_month'] = df['Delivery Date'].dt.month
  df['dt_year'] = df['Delivery Date'].dt.year
  df['dt_weekday'] = df['Delivery Date'].dt.dayofweek
  df.drop('Delivery Date', axis=1, inplace=True)
  return df

In [125]:
train_df_processed = extract_date(train_df_processed)
train_df_processed.head(2)

,Artist Reputation,Height,Width,Weight,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Fragile,Customer Information,Remote Location,Cost,Material_1,Material_2,Material_3,Material_4,Material_5,Material_6,Transport_1,Transport_2,state,date_diff,dt_day,dt_month,dt_year,dt_weekday
0,0.26,17.0,6.0,4128.0,13.91,16.27,1,1,0,0,1,0,-283.29,1,0,0,0,0,0,0,0,35,4,3,6,2015,2
1,0.28,3.0,3.0,61.0,6.83,15.00,0,0,0,0,1,0,-159.96,1,0,0,0,0,0,1,0,50,1,5,3,2017,6


In [126]:
test_df_processed = extract_date(test_df_processed)
test_df_processed.head(2)

,Artist Reputation,Height,Width,Weight,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Fragile,Customer Information,Remote Location,Material_1,Material_2,Material_3,Material_4,Material_5,Material_6,Transport_1,Transport_2,state,date_diff,dt_day,dt_month,dt_year,dt_weekday
0,0.35,53.0,18.0,871.0,5.98,19.11,1,1,0,0,1,0,0,0,0,0,0,1,0,0,12,-3,6,7,2017,3
1,0.67,7.0,4.0,108.0,6.92,13.96,0,0,0,1,1,0,0,0,1,0,0,0,1,0,35,0,2,5,2016,0


# **Initialializing Target and Features**

In [141]:
X = train_df_processed.drop('Cost', axis=1)
Y = train_df_processed['Cost']
Y.head()

0   -283.29
1   -159.96
2   -154.29
3   -161.16
4   -159.23
Name: Cost, dtype: float64

In [139]:
def log1p(target):
  return np.log1p(abs(target))

def expm1(x):
  return np.expm1(x)

def clipExp(vec):
    return np.clip(expm1(vec), 0, None)

In [143]:
# Applying log on target feature with abs as it has many negative values
Y_log = Y.apply(log1p)
Y_log.min()

4.466712392840524

# **Splitting Training and validation dataset**

In [144]:
x_train, x_valid , y_train, y_valid = train_test_split(X,Y_log, test_size=0.2, random_state=42)
x_train.shape, y_train.shape,x_valid.shape, y_valid.shape

((5200, 26), (5200,), (1300, 26), (1300,))

# **Feature Scaling**

In [148]:
scaler = MinMaxScaler()

# Apply scaler to train_variables
x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)

# Apply scaler to test_variables
x_valid_scaled = pd.DataFrame(scaler.transform(x_valid), columns=x_valid.columns)

In [149]:
x_train_scaled.describe()

,Artist Reputation,Height,Width,Weight,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Fragile,Customer Information,Remote Location,Material_1,Material_2,Material_3,Material_4,Material_5,Material_6,Transport_1,Transport_2,state,date_diff,dt_day,dt_month,dt_year,dt_weekday
count,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000,5200.000000
mean,0.460362,0.267876,0.155709,0.003065,0.003024,0.307039,0.339038,0.335385,0.395577,0.160385,0.731154,0.175577,0.245000,0.128077,0.126154,0.127500,0.117692,0.125192,0.532692,0.188269,0.541199,0.506635,0.483968,0.474213,0.459423,0.498942
std,0.249721,0.167413,0.108171,0.021938,0.022607,0.299780,0.473428,0.472170,0.489021,0.366998,0.443402,0.380496,0.430129,0.334208,0.332055,0.333564,0.322274,0.330969,0.498978,0.390965,0.316168,0.305377,0.292494,0.307392,0.337640,0.332521
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.270000,0.142857,0.083333,0.000005,0.000006,0.074989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.274510,0.250000,0.233333,0.181818,0.250000,0.166667
50%,0.450000,0.242857,0.125000,0.000026,0.000014,0.150256,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.549020,0.500000,0.466667,0.454545,0.500000,0.500000
75%,0.640000,0.371429,0.187500,0.000226,0.000232,0.538342,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.823529,0.750000,0.733333,0.727273,0.750000,0.833333
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# **Building Model**

In [ ]:
# rf_model = RandomForestRegressor(n_estimators=1000, )
param_test1 = {
    'n_estimators' : [10,50, 100, 500, 1000],
    'max_depth' : np.arange(2,12, 2)
}
rf_cv1 = GridSearchCV(estimator=RandomForestRegressor(criterion='mse', random_state=1), param_grid=param_test1, verbose=2, scoring='neg_mean_squared_error', cv=5, return_train_score=True)
rf_cv1.fit(x_train_scaled, y_train)


In [156]:
print( "Best parameters: ", rf_cv1.best_params_ , "Best Score:",  rf_cv1.best_score_)

Best parameters:  {'max_depth': 10, 'n_estimators': 1000} Best Score: -0.12043547926768809


In [158]:
params_grid2 = {
    'min_samples_split' : np.arange(2,12,3),
    'min_samples_leaf' : np.arange(1,10,3)
}

rf_cv2 = GridSearchCV(estimator=RandomForestRegressor(criterion='mse', n_estimators=1000, max_depth=10), param_grid = params_grid2, cv =5, verbose=1, scoring='neg_mean_squared_error', n_jobs=-1)
rf_cv2.fit(x_train_scaled, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 14.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=10,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=1000, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_job

In [159]:
print( "Best parameters: ", rf_cv2.best_params_ , "Best Score:",  rf_cv2.best_score_)

Best parameters:  {'min_samples_leaf': 4, 'min_samples_split': 8} Best Score: -0.12003589918039181


In [160]:
rf_model = RandomForestRegressor(n_estimators=1000, max_depth=10, min_samples_split= 8 , min_samples_leaf=4)
rf_model.fit(x_train_scaled, y_train)  

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=4,
                      min_samples_split=8, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [167]:
y_pred = rf_model.predict(x_train_scaled)
mean_squared_error(y_train, y_pred)

0.05010800936567998

In [168]:
y_valid_pred = rf_model.predict(x_valid_scaled)
mean_squared_error(y_valid, y_valid_pred)

0.12136428451857113

In [169]:
x_test_scaled = scaler.transform(test_df_processed)

In [170]:
test_predictions = rf_model.predict(x_test_scaled)

In [172]:
type(test_predictions)

numpy.ndarray

In [173]:
test_predictions = np.ravel(test_predictions)

In [174]:
test_predictions = clipExp(test_predictions)

In [175]:
test_predictions

array([   220.87267642,    289.30052586,    215.68787265, ...,
          527.44244787,    237.13448241, 183650.10236192])

In [176]:
submission =  pd.DataFrame({'Customer Id' : test_original['Customer Id'] , 'Cost': test_predictions})

In [178]:
submission.to_csv('submission_rf.csv')